In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define directories
train_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train/BTTAIxNYBG-train'
test_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test/BTTAIxNYBG-test'
validation_dir = '/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation/BTTAIxNYBG-validation'

# Load datasets
train_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-train.csv')
test_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-test.csv')
validate_df = pd.read_csv('/kaggle/input/bttai-nybg-2024/BTTAIxNYBG-validation.csv')

# Data augmentation configuration for training
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    rescale=1./255
)

# Note: No augmentation for validation and test data, only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Convert dataframe to a format suitable for the model training
def df_to_dataset(dataframe, datagen, directory, batch_size=32):
    return datagen.flow_from_dataframe(
        dataframe=dataframe,
        directory=directory,
        x_col='imageFile',  # Column in dataframe that contains the filenames
        y_col='classLabel',  # Column in dataframe that contains the class/label
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical'  # Change this if not a multiclass classification
    )

# Create datasets for training, validation, and testing
train_dataset = df_to_dataset(train_df, train_datagen, train_dir)
validation_dataset = df_to_dataset(validate_df, validation_datagen, validation_dir)

# Load the MobileNetV2 model, pretrained on ImageNet, without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Unfreeze some layers for fine-tuning
fine_tune_at = 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# Create the model
model = Sequential([
    base_model,
    # Convert features to vectors
    tf.keras.layers.GlobalAveragePooling2D(),
    # Add a dense layer for classification
    Dense(1024, activation='relu'),
    # Final layer with softmax activation for multi-class classification
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Start with a small portion (e.g., 10%) of the training data
train_subset = train_df.sample(frac=0.6, random_state=42)
train_subset_dataset = df_to_dataset(train_subset, train_datagen, train_dir)

# Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
]

# Fine-tune the model
history = model.fit(
    train_subset_dataset,  # Use the subset of data
    validation_data=validation_dataset,
    epochs=2,  # Initially train for fewer epochs for debugging
    callbacks=callbacks
)

# Now, fine-tune the model with the whole training dataset
# Modify the parameters according to your specific needs
# For example, increase the number of epochs and adjust the learning rate
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=3,  # Train for more epochs for fine-tuning
    callbacks=callbacks
)

validation_loss, validation_accuracy = model.evaluate(validation_dataset)
print(f'Validation Loss: {validation_loss}')
print(f'Validation Accuracy: {validation_accuracy}')

# Testing
test_dataset = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=test_dir,
    x_col='imageFile',  # Make sure this column name matches your test_df column name for filenames
    target_size=(224, 224),
    batch_size=32,
    class_mode=None,  # No labels
    shuffle=False
)
predictions = model.predict(test_dataset)
predicted_class_indices = np.argmax(predictions, axis=1)

# Save predictions to a CSV file
submission_df = pd.DataFrame({'uniqueID': test_df['uniqueID'], 'classID': predicted_class_indices})
submission_df.to_csv('/kaggle/working/submission.csv', index=False)


2024-03-31 16:01:53.460573: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-31 16:01:53.460674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-31 16:01:53.579433: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 81946 validated image filenames belonging to 10 classes.
Found 10244 validated image filenames belonging to 10 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Found 49168 validated image filenames belonging to 10 classes.
Epoch 1/2


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


   1/1537 ━━━━━━━━━━━━━━━━━━━━ 13:16:13 31s/step - accuracy: 0.1875 - loss: 2.3392

I0000 00:00:1711901115.254257      81 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1537/1537 ━━━━━━━━━━━━━━━━━━━━ 1242s 788ms/step - accuracy: 0.8605 - loss: 0.4276 - val_accuracy: 0.9109 - val_loss: 0.3110
Epoch 2/2
1537/1537 ━━━━━━━━━━━━━━━━━━━━ 845s 548ms/step - accuracy: 0.9492 - loss: 0.1487 - val_accuracy: 0.9406 - val_loss: 0.1814
Restoring model weights from the end of the best epoch: 2.
Epoch 1/3
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 1642s 639ms/step - accuracy: 0.9533 - loss: 0.1320 - val_accuracy: 0.9662 - val_loss: 0.1062
Epoch 2/3
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 1669s 650ms/step - accuracy: 0.9639 - loss: 0.1028 - val_accuracy: 0.9552 - val_loss: 0.1486
Epoch 3/3
2561/2561 ━━━━━━━━━━━━━━━━━━━━ 1565s 610ms/step - accuracy: 0.9674 - loss: 0.0911 - val_accuracy: 0.9638 - val_loss: 0.1297
Restoring model weights from the end of the best epoch: 1.
321/321 ━━━━━━━━━━━━━━━━━━━━ 81s 254ms/step - accuracy: 0.9670 - loss: 0.1024
Validation Loss: 0.10727931559085846
Validation Accuracy: 0.9662241339683533
Found 30690 validated image filenames.
960/960 ━━━━━━━━━━━━━━━━━━━━ 